In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install konlpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 54.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.3/465.3 kB 37.0 MB/s eta 0:00:00


## 1. Data Load

In [ ]:
import pandas as pd
from konlpy.tag import Okt
from string import whitespace, punctuation
import re, unicodedata

In [ ]:
path= '/content/drive/My Drive/23 U 4-1/텍스트마이닝/기말프로젝트/Data/'
df= pd.read_csv(path+ 'train_data.csv', encoding= 'utf-8-sig')
df.head()

,Unnamed: 0,newsTitle,newsSubTitle,newsContent,newsCategory,newsSubcategory,sentenceCount,clickbaitClass,clickbaitTitle
0,0,"[오늘의 SR이슈] 윤리 문제 불거진 종근당 회장 조사 마치고 귀가…\""사죄한다\""",NaN,"\""사죄한다.\"" 운전기사들에게 허구한 날 폭언을 퍼부어 윤리 문제가 불거진 이장한...",경제,일반,9,0,'친환경 금융 목표'…'탄소중립' 실천 나선 금융업계
1,1,"최종구 금융위원장, 임팩트금융 정착 위해 정부 차원 기금 조성",NaN,"최종구 금융위원회 위원장이 \""'임팩트금융(사회적금융)'의 안착 차원에서 정부가 직...",경제,은행,13,0,공동주택 하자분쟁 조정 본격화
2,2,한국기업지배구조원 733개 기업 ESG 등급 부여,NaN,"한국기업지배구조원이 8월 31일 국내 상장회사의 환경경영(E), 사회책임경영(S),...",경제,은행,13,0,"LG생활건강, 사상 최대 실적에도 주가 반등은 '글쎄'"
3,3,가을 맞아 더 활발한 기업 프로보노 활동,NaN,요즘 프로보노 형 사회 공헌 활동을 펴는 회사가 늘고 있다.\n회사를 경영하거나 기...,경제,산업,13,0,"로봇청소기 강자 코오롱글로벌, '드리미 W10' 국내 론칭"
4,4,2017 다우존스 지속가능경영지수 어떤 기업이 편입되었나?,NaN,"런던 현지 시각으로 7일, 스탠다드앤푸어스 다우존스 인디시즈와 로베코샘이 다우존스지...",경제,일반,9,0,"한숨만 쉬는 유통업계, 매출 얼마나 줄었나 봤더니…"


In [ ]:
# Unnecessary Columns Drop
df.drop(['Unnamed: 0', 'newsSubTitle', 'newsSubcategory', 'sentenceCount'], axis=1, inplace= True)
df.columns= ['title', 'content',
       'newsCategory', 'clickbaitClass',
       'clickbaitTitle']

In [ ]:
# White space, punctuation removal
pattern_whitespace = re.compile(f'[{whitespace}]+')
df['title'] = df.title.fillna('').replace(pattern_whitespace, ' ').map(lambda x: unicodedata.normalize('NFC', x)).str.strip()
df['clickbaitTitle'] = df.clickbaitTitle.fillna('').replace(pattern_whitespace, ' ').map(lambda x: unicodedata.normalize('NFC', x)).str.strip()
df['content'] = df.content.str.replace(pattern_whitespace, ' ').map(lambda x: unicodedata.normalize('NFC', x)).str.strip()

def CleanEnd(text):
    email = re.compile(r'[-_0-9a-z]+@[-_0-9a-z]+(?:\.[0-9a-z]+)+', flags=re.IGNORECASE)
    url = re.compile(r'(?:https?:\/\/)?[-_0-9a-z]+(?:\.[-_0-9a-z]+)+', flags=re.IGNORECASE)
    etc = re.compile(r'\.([^\.]*(?:기자|특파원|교수|작가|대표|논설|고문|주필|부문장|팀장|장관|원장|연구원|이사장|위원|실장|차장|부장|에세이|화백|사설|소장|단장|과장|기획자|큐레이터|저작권|평론가|©|©|ⓒ|\@|\/|=|▶|무단|전재|재배포|금지|\[|\]|\(\))[^\.]*)$')
    bracket = re.compile(r'^((?:\[.+\])|(?:【.+】)|(?:<.+>)|(?:◆.+◆)\s)')
    
    result = email.sub('', text)
    result = url.sub('', result)
    result = etc.sub('.', result)
    result = bracket.sub('', result).strip()
    return result
df['title'] = df.title.map(CleanEnd)
df['clickbaitTitle'] = df.clickbaitTitle.map(CleanEnd)
df['content'] = df.content.map(CleanEnd)

def TextFilter(text):
    punct = ''.join([chr for chr in punctuation if chr != '%'])
    filtering = re.compile(f'[{whitespace}{punct}]+')
    onlyText = re.compile(r'[^\% ㄱ-ㅣ가-힣]+')
    result = filtering.sub(' ', text)
    result = onlyText.sub(' ', result).strip()
    result = filtering.sub(' ', result)
    return result
df['title'] = df.title.map(TextFilter)
df['clickbaitTitle'] = df.clickbaitTitle.map(TextFilter)
df['content'] = df.content.map(TextFilter)


In [ ]:
# Noun Extraction
okt= Okt()
lzNounTitle= []
lzNounCTitle= []
lzNounContent= []

for i in range(len(df)): 
  lzNounTitle.append(okt.nouns(df.iloc[i,0]))
  lzNounContent.append(okt.nouns(df.iloc[i,1]) )
  lzNounCTitle.append(okt.nouns(df.iloc[i,4]))
df['title']= lzNounTitle
df['content']= lzNounContent
df['clickbaitTitle']= lzNounCTitle
df.head()

,title,content,newsCategory,clickbaitClass,clickbaitTitle
0,"[윤리, 문제, 종근당, 회장, 조사, 마치, 귀가, 사죄]","[사죄, 운전기사, 허구, 날, 폭언, 퍼, 부어, 윤리, 문제, 이장, 종근당, ...",경제,0,"[친환경, 금융, 목표, 탄소, 중립, 실천, 나선, 금융업]"
1,"[최종, 구, 금융, 위원장, 임팩트, 금융, 정착, 위해, 정부, 차원, 기금, 조성]","[최종, 구, 금융, 위원회, 위원장, 임팩트, 금융, 사회, 금융, 의, 안착, ...",경제,0,"[공동, 주택, 하자, 분쟁, 조정, 본격]"
2,"[한국, 기업, 지배구조, 개, 기업, 등급, 부여]","[한국, 기업, 지배구조, 월, 일, 국내, 상장회사, 환경, 경영, 사회, 책임,...",경제,0,"[생활, 건강, 사상, 최대, 실적, 주가, 반등, 글쎄]"
3,"[가을, 더, 기업, 프로보노, 활동]","[요즘, 프로보노, 형, 사회, 공헌, 활동, 회사, 늘, 회사, 경영, 거나, 기...",경제,0,"[로봇청소기, 강, 코오롱, 글로벌, 국내, 론칭]"
4,"[다우존스, 가능, 지수, 기업, 편입]","[런던, 현지, 시각, 일, 스탠다드, 앤, 어스, 다우존스, 인, 디시, 와, 로...",경제,0,"[한숨, 유통업, 매출, 얼마나]"


In [ ]:
df.isna().sum()

title             0
content           0
newsCategory      0
clickbaitClass    0
clickbaitTitle    0
dtype: int64

In [ ]:
df.to_csv('TM_train_pp.csv', encoding= 'utf-8-sig', index= False)